## Multiprocessing

Threading vs Multiprocessing

The threading module uses threads, the multiprocessing module uses processes. The difference is that threads run in the same memory space, while processes have separate memory. This makes it a bit harder to share objects between processes with multiprocessing. Since threads use the same memory, precautions have to be taken or two threads will write to the same memory at the same time. This is what the global interpreter lock is for.

Spawning processes is a bit slower than spawning threads.

Another option would be Ray.

From: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python

In [1]:
from multiprocessing import Lock, Process, Value, Pool
import time
import numpy as np

In [171]:
def add(total, lock):
    for _ in range(1000):
        time.sleep(0.001)
        lock.acquire()
        total.value += 20
        lock.release()
    
def sub(total, lock):
    for _ in range(100):
        time.sleep(0.001)
        lock.acquire()
        total.value -= 20
        lock.release()

#### The lock makes sure that no other thread can write to this value before releasing it.

In [180]:
total = Value('i', 500) # i for integer

lock = Lock()

In [181]:
add_p = Process(target=add, args=(total,lock))
sub_p = Process(target=sub, args=(total,lock))

In [182]:
add_p.start()
sub_p.start()

add_p.join()
sub_p.join()

In [183]:
total.value

18500

In [2]:
def f():
    for i in range(10000):
        for j in range(10000):
            x = i*j
    return 10

In [24]:
def m(): return np.random.randn(100,100,100)

In [4]:
pool = Pool()

### Spawn f() on 3 cores

Look at htop to be sure!

In [30]:
%%time

result1 = pool.apply_async(f)
result2 = pool.apply_async(f)
result3 = pool.apply_async(f)
result4 = pool.apply_async(f)

result1.get()
result2.get()
result3.get()
result4.get()

CPU times: user 3.03 ms, sys: 90 µs, total: 3.12 ms
Wall time: 5.14 s


### Run f() on 1 core after on another

In [31]:
%%time

f()
f()
f()
f()

CPU times: user 17.4 s, sys: 17.4 ms, total: 17.4 s
Wall time: 17.4 s


### Spawn m() on 4 cores

In [29]:
%%time

result1 = pool.apply_async(m)
result2 = pool.apply_async(m)
result3 = pool.apply_async(m)

result1.get()
result2.get()
result3.get()

CPU times: user 2.31 ms, sys: 1.89 ms, total: 4.19 ms
Wall time: 4.06 s


In [26]:
%%time

m()
m()
m()

CPU times: user 133 ms, sys: 3.92 ms, total: 137 ms
Wall time: 135 ms


## Wow this is way slower... 

I think multiprocessing is great for making "normal" python code faster. To make somethin like single-threaded numpy function faster seems to be hard.